In [1]:
import os
import importlib
path='/home/enis/projects/nna/src/scripts/'
os.chdir(path)

import teacher

import itertools
import pandas as pd

import glob

import numpy as np

from pathlib import Path
import csv
import random
import datetime

Output(layout=Layout(border='1px solid black'))

In [2]:
importlib.reload(teacher)

version_tag='weather_V1'
config = teacher.setup(versiontag=version_tag)

weather_data_folder='/scratch/enis/data/nna/weather_data/2017_2020'

root_path = '/scratch/enis/data/nna/labeling/samples'
config['split_out_path'] = f'{root_path}/{version_tag}/audio_'

## data-1: as many sampes as possible
config['new_dataset_path'] = f'{root_path}/{version_tag}/{version_tag}.csv'
config['dataset_version'] = 'W1'
# config['versiontag'] = 'yfitloiq-V1'
config['excell_label_headers']=['day_length','air_temp','snow_depth',
        'cloud_fraction','relative_humidity','runoff','rain_precip',
         'snow_precip','wind_direction','wind_speed']

# config['excell_label_headers']=['day_length','air_temp','snow_depth',
#         'cloud_fraction','relative_humidity','runoff','rain_precip',
#          'snow_precip','wind_direction','wind_speed',
#         'snow_blowing_ground','snow_blowing_air',]
        


config['upper_taxo_links']={}

FILE_PER_LOCATION=200
# print('total sample count to be produced:',file_per_location*len(short_ones.keys()))

# 40 Prudhoe or ANWR monitoring sites AND the Ivvavik sites
short_input_csv_headers = ['day_length','air_temp','snow_depth',
        'cloud_fraction','relative_humidity','runoff','rain_precip',
         'snow_precip','wind_direction','wind_speed']
# (year,month,day,hour,day_length,air_temp,snow_depth,
#                 cloud_fraction,relative_humidity,runoff,rain_precip,
#                     snow_precip,wind_direction,wind_speed)=row
# for Dalton and Dempster
long_input_csv_headers = ['day_length','air_temp','snow_depth',
        'cloud_fraction','relative_humidity','runoff','rain_precip',
         'snow_precip','total_precip','wind_direction','wind_speed',
        'snow_blowing_ground','snow_blowing_air']


# (year,month,day,hour,day_length,air_temp,snow_depth,
# cloud_fraction,relative_humidity,runoff,rain_precip,
# snow_precip,total_precip,wind_direction,wind_speed,
# snow_blowing_ground,snow_blowing_air)=row

short_locations=('prudhoe','ivvavik','anwr')
long_locations=('dalton','dempster')
    


In [3]:
file_properties_df = pd.read_pickle(
    '/scratch/enis/data/nna/database/allFields_dataV10.pkl')


In [4]:

def csv_path_per_regloc(data_folder):
    
    station_csv={}
    for region_path in glob.glob(f'{data_folder}/*'):
        locations=glob.glob(region_path+'/sm_products_by_station/*')
        region = Path(region_path).name.lower()
        for location_path in locations:
            location=Path(location_path).stem.split('_')[-1]
            if region!=location[:-2].lower():
                print(region,location)
            location=location[-2:]
            # print(location_path)
            if region=='ivvavik':
                location='SINP'+location
            station_csv[(region,location)]=location_path
            # print(region,location)
        # print(len(locations))
    return station_csv

def year_per_regloc(station_csv,file_properties_df):

    station_years={}
    for region,location in station_csv.keys():
        region_filtered=file_properties_df[file_properties_df['region']==region]
        loc_reg_filtered = region_filtered[region_filtered['locationId']==location]
        
        # print(region,location)
        unique_years = (loc_reg_filtered.year.unique())
        unique_years = [int(year) for year in unique_years if int(year)>2018]
        # print(unique_years)
        station_years[(region,location)] = unique_years
    return station_years


In [5]:
station_csv = csv_path_per_regloc(weather_data_folder)
station_years = year_per_regloc(station_csv,file_properties_df)


In [6]:
# region='dalton'
# location='10'
# region_filtered=file_properties_df[file_properties_df['region']==region]
# loc_reg_filtered = region_filtered[region_filtered['locationId']==location]
# loc_reg_filtered # 2019-07-23_10:30:00

In [7]:
def load_rows(csv_fname,region,short_locations,long_locations):
    with open(csv_fname, newline='') as csvfile:
        # reader = csv.DictReader(csvfile,fieldnames=fieldnames)
        csv_reader = list(csv.reader(csvfile))

        short = True if len(csv_reader[0])==14 else False
        if region in short_locations and not short:
            raise Exception('short location has long csv {},{}'.format(region,location))
        if region in long_locations and short:
            raise Exception('long location has short csv {},{}'.format(region,location))

        return csv_reader,short

def get_random_rows(reader,file_per_location,station_years):
        reader = [row for row in reader if int(row[0]) in station_years[(region,location)]]
        rows_picked = random.choices(reader,k=file_per_location)
        return rows_picked

def parse_rows(rows_picked,location,region,short,short_headers,long_headers):

    input_csv_headers = short_headers if short else long_headers

    pd_rows=[]
    for row in rows_picked:
        year, month, day, hour = [int(row[x]) for x in range(4)]

        pd_row={}
        pd_row['location']=location
        pd_row['region']=region.lower()
        timestamp = datetime.datetime(year, month, day, hour=hour)
        # timestamps represent timeperiod starting from 3 hours earlier
        # we place them in the middle for finding best representation in audio
        timestamp = timestamp - datetime.timedelta(hours=1.5)
        pd_row['TIMESTAMP']= timestamp.strftime('%Y-%m-%d_%H:%M:%S')
        row[4:] = [float(x) for x in row[4:]]
        for label,data in zip(input_csv_headers,row[4:]):
            pd_row[label]=data
        pd_rows.append(pd_row)
    return pd_rows




In [8]:
# # 2019-06-10_07:30:00

# filtered=file_properties_df[file_properties_df['region']=='dalton']
# filtered = filtered[filtered['locationId']=='10']
# filtered=filtered[filtered['timestamp']> datetime.datetime(2019,6,17,0,30,0)]
# filtered=filtered[filtered['timestamp']< datetime.datetime(2019,6,18,0,0,0)]
# filtered

In [ ]:
dataset_csv=[]
missing_csv=[]
for (region,location),fname in station_csv.items():
    csv_reader,short = load_rows(fname,region,short_locations,long_locations)
    rows_picked = get_random_rows(csv_reader,FILE_PER_LOCATION,station_years)

    pd_rows = parse_rows(rows_picked,location,region,short,short_input_csv_headers,long_input_csv_headers)


    selected_data=pd.DataFrame.from_dict(pd_rows)
    new_dataset_csv, not_found_rows = teacher.generate_new_dataset(
        selected_data,
        config['versiontag'],
        config['split_out_path'],
        file_properties_df,
        config['upper_taxo_links'],
        config['dataset_version'],
        buffer=3600,
        excell_label_headers= {},#config['excell_label_headers'],
        labels_thresholds= {},#config['labels_thresholds'],
        outputSuffix='.wav',
        dry_run=False,
        excell_labels_2_names= {}, #config['excell_labels_2_names'],
        stereo2mono=True,
        overwrite=False,
        sampling_rate=48000,
        label_row_by_threshold=False,
    )
    dataset_csv.extend(new_dataset_csv)
    missing_csv.extend(not_found_rows)


# from copy import deepcopy
# new_dataset_csv_backup = deepcopy(new_dataset_csv)
# not_found_rows_backup = deepcopy(not_found_rows)

# del not_found_rows_backup,new_dataset_csv_backup


# write_csv(config['new_dataset_path'],
#           new_dataset_csv,
#           fieldnames=config['excell_all_headers'])

In [24]:
len(dataset_csv)

3811

In [29]:
from copy import deepcopy
dataset_csv_backup = deepcopy(dataset_csv)
missing_csv_backup = deepcopy(missing_csv)

excell_all_headers = [
    'data_version', 'Annotator', 'Site ID', 'File Name', 'Date', 'Start Time',
    'End Time', 'Length', 'Clip Path', 'Comments', 'weather_timestamp',
    'region', 'day_length', 'air_temp', 'snow_depth', 'cloud_fraction',
    'relative_humidity', 'runoff', 'rain_precip', 'snow_precip',
    'wind_direction', 'wind_speed'
]

for row in dataset_csv_backup:
    for k,v in row['weather_data'].items():
        if k not in ['location','TIMESTAMP']:
            row[k]=v
        row['weather_timestamp'] = row['weather_data']['TIMESTAMP']
    del row['weather_data']


for row in dataset_csv_backup:
    for k in list(row.keys()):
        if k not in excell_all_headers:
            del row[k]

dataset_csv_unique = {}
for row in dataset_csv_backup:
    dataset_csv_unique[row['Clip Path']] = row
dataset_csv_backup=list(dataset_csv_unique.values())

print(len(dataset_csv_backup))
teacher.write_csv(config['new_dataset_path'],
          dataset_csv_backup,
          fieldnames=excell_all_headers)

3700


In [30]:
dataset_csv_backup[0]

{'data_version': 'W1',
 'Annotator': 'weather_V1',
 'Site ID': 'SINP03',
 'File Name': '/tank/data/nna/real/ivvavik/SINP03/2019/SINP-03_20190513_231602.flac',
 'Start Time': '00:29:50.000000',
 'End Time': '00:30:00.000000',
 'Date': '05/14/2019',
 'Length': '00:00:10.000000',
 'Clip Path': PosixPath('/scratch/enis/data/nna/labeling/samples/weather_V1/audio_weather_V1/ivvavik/SINP03/SINP-03_20190513_231602_73m_48s__73m_58s.wav'),
 'Comments': '',
 'weather_timestamp': '2019-05-14_01:30:00',
 'region': 'ivvavik',
 'day_length': 20.2996368,
 'air_temp': 0.922241211,
 'snow_depth': 0.450530827,
 'cloud_fraction': 0.831612885,
 'relative_humidity': 84.5013275,
 'runoff': 0.0,
 'rain_precip': 3.24604343e-06,
 'snow_precip': 4.02112619e-06,
 'wind_direction': 291.185028,
 'wind_speed': 1.36716318}

In [68]:
len(dataset_csv)

3935

In [12]:
a=[row['Clip Path'] for row in dataset_csv]

In [13]:
len((a)),len(set(a))

(3811, 3700)

In [83]:
for row in dataset_csv:
    m = row['Clip Path']
    if m.exists():
        continue
    else:
        print(row)
        break

{'data_version': 'W2', 'Annotator': 'weather_V2', 'Site ID': '03', 'File Name': '/tank/data/nna/real/dalton/03/2019/S4A10437_20191009_172000.aac', 'Start Time': '17:20:00.000000', 'End Time': '17:20:10.000000', 'Date': '10/09/2019', 'Length': '00:00:10.000000', 'Clip Path': PosixPath('/scratch/enis/data/nna/labeling/samples/weather_V2/audio_weather_V2/dalton/03/S4A10437_20191009_172000_0m_0s__0m_10s.wav'), 'Comments': '', 'weather_timestamp': '2019-10-09_16:30:00', 'region': 'dalton', 'day_length': 9.85344791, 'air_temp': -0.127105713, 'snow_depth': 0.0195905175, 'cloud_fraction': 0.831612885, 'relative_humidity': 95.3483429, 'runoff': 0.0, 'rain_precip': 1.14912109e-05, 'snow_precip': 7.56504087e-05, 'total_precip': 8.71416196e-05, 'wind_direction': 192.373138, 'wind_speed': 1.04121852, 'snow_blowing_ground': 0.0, 'snow_blowing_air': 0.0787686035}
